[Quelle:](https://rolisz.ro/2013/04/18/neural-networks-in-python/ "hier")

In [23]:
import numpy as np
# die Aktivierungsfunktion und ihre erste Ableitung
def tanh(x):
    return np.tanh(x)
def tanh_prime(x):
    return 1.0-np.tanh(x)**2
class NeuralNetwork:
    def __init__(self, layers, activation='tanh'):
        if activation == 'sigmoid':
            self.activation = sigmoid
            self.activation_prime = sigmoid_prime
        elif activation == 'tanh':
            self.activation = tanh
            self.activation_prime = tanh_prime
        # Set weights
        self.layers=layers
        self.weights = []        
        for i in range(1, len(layers) - 1):
            r = 2*np.random.random((layers[i-1] + 1, layers[i] + 1)) -1            
            self.weights.append(r)
        # output layer - random((2+1, 1)) : 3 x 1
        r = 2*np.random.random( (layers[i] + 1, layers[i+1])) - 1
        self.weights.append(r)
        
    def bestimmeGewichte(self,wert):
        self.weights=[]
        layers=self.layers
        for i in range(1, len(layers) - 1):
            r = np.zeros((layers[i-1] + 1, layers[i] + 1)) +wert
            #print(i,r)
            self.weights.append(r)
        r = np.zeros( (layers[i] + 1, layers[i+1])) +wert
        self.weights.append(r)
        print("feste Gewichte\n",self.weights,self.weights[1].shape)
        
    def prepareInput(self,inListe):
        # setzt eine Spalte mit Einsen als Bias vor die Eingabe
        eingabe=inListe#np.array(inListe)
        X        = np.ones((len(eingabe),len(eingabe[0])+1))
        X[:,1:] = eingabe
        return X
    
    def fit(self, X, y, learning_rate=0.2, epochs=100000):        
        self.fehler=0        
        for k in range(epochs):                
            i = np.random.randint(X.shape[0])
            a = [X[i]]            
            for l in range(len(self.weights)):
                    dot_value = np.dot(a[l], self.weights[l])
                    activation = self.activation(dot_value)                    
                    a.append(activation)                    
            error = y[i] - a[-1]
            self.fehler+=np.sum(np.square(error)) #Summe der Quadrate     
            deltas = [error * self.activation_prime(a[-1])]

            # we need to begin at the second to last layer 
            # (a layer before the output layer)
            for l in range(len(a) - 2, 0, -1): 
                deltas.append(deltas[-1].dot(self.weights[l].T)*self.activation_prime(a[l]))
            # reverse
            # [level3(output)->level2(hidden)]  => [level2(hidden)->level3(output)]
            deltas.reverse()            
            # backpropagation
            # 1. Multiply its output delta and input activation 
            #    to get the gradient of the weight.
            # 2. Subtract a ratio (percentage) of the gradient from the weight.
            for i in range(len(self.weights)):
                layer=np.array(a[i])
                layer=layer[:,None] #Umwandeln in eine nx1 Matrix 
                delta=np.array(deltas[i])
                delta=delta[None,:] #Umwandeln in eine 1xn-Matrix               
                self.weights[i]+=learning_rate*np.dot(layer,delta)
            

    def predict(self, x):        
        for l in range(0, len(self.weights)):
            x = self.activation(np.dot(x, self.weights[l]))
        return x


In [25]:

from sklearn.cross_validation import train_test_split 
from sklearn.datasets import load_digits
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelBinarizer
#from NeuralNetwork import NeuralNetwork

digits = load_digits()
X = digits.data
y = digits.target
X -= X.min() # normalize the values to bring them into the range 0-1
X /= X.max()

nn = NeuralNetwork([64,100,10])
X=nn.prepareInput(X)
X_train, X_test, y_train, y_test = train_test_split(X, y)
labels_train = LabelBinarizer().fit_transform(y_train)
labels_test = LabelBinarizer().fit_transform(y_test)

nn.fit(X_train,labels_train,epochs=30000)
predictions = []
for i in range(X_test.shape[0]):
    o = nn.predict(X_test[i] )
    predictions.append(np.argmax(o))
print (confusion_matrix(y_test,predictions))
print (classification_report(y_test,predictions))

[[44  0  0  0  1  0  2  0  1  1]
 [ 0 26  0  0  0  2  3  0 10  1]
 [ 0  3 32  1  0  2  2  0  2  0]
 [ 0  0  1 31  0  3  4  0  3  1]
 [ 0  0  0  0 43  0  1  3  0  0]
 [ 0  0  0  0  0 49  1  0  0  1]
 [ 0  0  0  0  0  0 44  0  1  0]
 [ 0  0  0  0  0  0  0 42  0  0]
 [ 0  0  0  0  0  4  4  0 35  1]
 [ 1  0  0  0  0  1  2  2  1 38]]
             precision    recall  f1-score   support

          0       0.98      0.90      0.94        49
          1       0.90      0.62      0.73        42
          2       0.97      0.76      0.85        42
          3       0.97      0.72      0.83        43
          4       0.98      0.91      0.95        47
          5       0.80      0.96      0.88        51
          6       0.70      0.98      0.81        45
          7       0.89      1.00      0.94        42
          8       0.66      0.80      0.72        44
          9       0.88      0.84      0.86        45

avg / total       0.87      0.85      0.85       450

